In [ ]:
from urllib.request import urlretrieve
# 資料集
url = "https://github.com/Elwing-Chou/tt0517/raw/master/pttexsmall.csv"
urlretrieve(url, "ptt.csv")
# jieba大辭典
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.jieba.big")
# ptt專用詞彙
url = "https://github.com/Elwing-Chou/tt0517/raw/master/ptt_dic.txt"
urlretrieve(url, "dict.ptt")

('dict.ptt', <http.client.HTTPMessage at 0x7f69aa394048>)

In [ ]:
import jieba
jieba.set_dictionary("dict.jieba.big")
jieba.load_userdict("dict.ptt")

Building prefix dict from /content/dict.jieba.big ...
Dumping model to file cache /tmp/jieba.ude312db176676e4f808d1d24e4092662.cache
Loading model cost 1.630 seconds.
Prefix dict has been built successfully.


In [ ]:
import pandas as pd
df = pd.read_csv("ptt.csv")
df

,type,title,content
0,問卦,小北若去日本展店 會被唐吉一拳打死嗎?,\n號稱台灣版驚安の殿堂的小北百貨\n\n24小時全年無休五金百貨生活百貨應有盡有\n\n在...
1,新聞,自排入不分區立委？ 李昭平：吳敦義從沒,\n※發文無1~6小標格式或未依順序任意刪除者會被刪文\n\n1.媒體來源:聯合報\n※ 例...
2,問卦,有沒有這波警察被桶覺青消失的八卦？,\n\n疑？阿不是只要有警察的新聞出現在八卦版\n八卦覺青就會高潮？\n每次不是都愛拿太陽花...
3,新聞,謝鑫佑被唬到大 小說寫覆鼎金多年後成預言,\n謝鑫佑被唬到大 小說寫覆鼎金多年後成預言\n\n（中央社記者鄭景雯台北3日電）「我跟馬奎...
4,新聞,促韓國瑜辭職 高市藍綠議員爆肢體衝突,\n※發文無1~6小標格式或未依順序任意刪除者會被刪文\n\n1.媒體來源:\n※ 例如蘋果...
...,...,...,...
19995,問卦,你各位會以碗就口嗎？？？,\n\n乳題，是這樣啦\n小弟現在出來吃飯，環顧四周\n以碗就口的包括我就只有兩個\n其他的...
19996,問卦,握推手腕到底可不可以彎阿？,\n\n\n各位巨巨大家晚安大家好\n小弟是習慣不彎喇\n就是往上推時手腕跟手臂成一直線\n...
19997,問卦,佐佐木美玲的中文程度到底如何？,\nhttps://imgur.com/Lm0FbuA.jpg\n\n佐佐木美玲小時候曾經住...
19998,問卦,欸欸欸 講話是不是一定要多XD比較好?,\n乳體\n\n是這樣的拉XD\n\n本肥宅常常看到有人在網路上亂嗆人\n\n結果就被對方吉...


In [ ]:
import re
def pttcut(content):
    content = re.sub(r'https?:\/\/.*[\r\n]*', '', content)
    punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…~/ －＊➜■─★☆=@<>◉é''')
    content = filter(lambda x: x not in punct, jieba.cut(content))
    return " ".join(content)
x_train = df["content"].apply(pttcut)
x_train

0        \n 號稱 台灣版 驚安 の 殿堂 的 小北 百貨 \n \n 24 小時 全年 無休 五金...
1        \n ※ 發文 無 1 6 小 標格 式 或 未 依 順序 任意 刪除 者 會 被刪 文 \...
2        \n \n 疑 阿 不是 只要 有 警察 的 新聞 出現 在 八卦 版 \n 八卦 覺青 就...
3        \n 謝鑫佑 被 唬 到 大 小說 寫 覆鼎金 多年 後 成 預言 \n \n 中央社 記者...
4        \n ※ 發文 無 1 6 小 標格 式 或 未 依 順序 任意 刪除 者 會 被刪 文 \...
                               ...                        
19995    \n \n 乳題 是 這樣 啦 \n 小弟 現在 出來 吃飯 環顧四周 \n 以碗 就 口 ...
19996    \n \n \n 各位 巨 巨大 家 晚安 大家 好 \n 小弟 是 習慣 不彎 喇 \n ...
19997    \n 佐佐木 美玲 小時候 曾經 住過 台灣 \n \n 也 曾 在 節目 中 透露 這段 ...
19998    \n 乳體 \n \n 是 這樣 的 拉 XD \n \n 本 肥宅 常常 看到 有人 在 ...
19999    \n 黑白棋 的 介面 在 Ptt 上面 應該 不難 設計 才 對 \n \n 日本 人 超...
Name: content, Length: 20000, dtype: object

In [ ]:
with open("ptt_cut.txt", "w", encoding="utf-8") as f:
    for post in x_train:
        f.write(post)
        f.write("\n")

In [ ]:
from gensim.models.word2vec import LineSentence, Word2Vec
sentences = LineSentence("ptt_cut.txt")
w2v = Word2Vec(sentences)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
print(len(w2v.wv.vocab))
print("肥宅" in w2v.wv.vocab)
print("Elwing" in w2v.wv.vocab)
w2v.wv["肥宅"]

41598
True
False


array([-0.18339424, -1.0691814 , -0.41155794,  0.8529014 ,  1.3975132 ,
        0.65376437,  0.04096989, -1.9908603 , -1.5362816 , -0.6587044 ,
       -1.6021143 , -3.3707576 ,  0.78839415, -1.1485138 ,  0.897239  ,
       -0.22457883,  0.23581837, -0.5680209 , -0.08846271,  0.13291264,
        0.09250135, -0.819456  ,  1.1601533 ,  0.60842943,  0.10182907,
        1.0743146 , -0.3093859 , -1.0314226 ,  1.0735426 ,  0.21244751,
        0.64625776,  0.6424596 ,  0.3596618 , -0.5950384 ,  1.3369725 ,
        0.9796453 , -0.96890485, -0.08009491, -0.76178265, -1.9759644 ,
       -0.65901816, -1.0528774 ,  1.6622936 , -2.5244522 ,  0.5924413 ,
       -1.212064  , -0.5260376 ,  2.0271306 , -0.57937145, -1.159247  ,
       -1.340982  ,  1.3989761 ,  0.08416298,  1.3871095 ,  1.0426493 ,
        0.8167228 ,  0.45490193, -1.8211112 ,  0.05476584, -1.3314179 ,
        0.46020937,  0.2321219 ,  0.49376887,  0.25434053,  2.2053835 ,
        0.5286489 , -0.99372715,  1.05799   ,  0.13153541, -0.19

In [ ]:
w2v.wv.most_similar("肥宅")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('肥肥', 0.8334802985191345),
 ('阿肥', 0.778662383556366),
 ('小妹', 0.7651031017303467),
 ('本肥', 0.7563671469688416),
 ('妹子', 0.748600959777832),
 ('魯蛇', 0.742835521697998),
 ('小弟', 0.7409191727638245),
 ('學長', 0.7239038944244385),
 ('烤肉', 0.7234455943107605),
 ('宅宅', 0.7215024828910828)]

In [ ]:
w2v.wv.similarity("肥宅", "妹子")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.7486009

In [ ]:
import numpy as np
import jieba.analyse
post_vecs = []
for post in x_train:
    tags = jieba.analyse.extract_tags(post, 
                                      topK=None, 
                                      withWeight=True)
    vec, ws = np.zeros((100, )), 0.0
    for k, tfidf in tags:
        if k in w2v.wv.vocab:
            vec = vec + w2v.wv[k] * tfidf
            ws = ws + tfidf
    post_vecs.append(vec/ws)
np.array(post_vecs).shape

(20000, 100)

In [ ]:
with open("post.txt", "w", encoding="utf-8") as f:
    f.write("{} {}\n".format(20000, 100))
    for i, v in enumerate(post_vecs):
        v = " ".join(map(str, v))
        f.write("{} {}\n".format(i, v))

In [ ]:
# 前面都是前置動作, 真正在用從這行開始
from gensim.models import KeyedVectors
d2v = KeyedVectors.load_word2vec_format("post.txt")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
idx = input("輸入代號[0-19999]:")
row = df.iloc[int(idx)]
print("原標題:", row["title"])
print("原文章:", row["content"])
print("-" * 100)
# 我只取第一篇[0]
idx, cos = d2v.most_similar(idx)[0]
print("cos距離:", cos)
row = df.iloc[int(idx)]
print("最像標題:", row["title"])
print("最像文章:", row["content"])

輸入代號[0-19999]:2
原標題: 有沒有這波警察被桶覺青消失的八卦？
原文章: 

疑？阿不是只要有警察的新聞出現在八卦版
八卦覺青就會高潮？
每次不是都愛拿太陽花圖片火力支援？

怎麼從昨天事情發生到現在
每篇都找不到覺青貼圖惹？？？

是不是覺青也會看風向？風頭不對就不敢po了？



神圖助攻了！
http://i.imgur.com/eXo260p.jpg



警察被捅，最開心的是不是當初去立法院外喝啤酒打卡拍照的那幾千名覺青呀？？？

-----
Sent from JPTT on my Xiaomi MI 9.

--

----------------------------------------------------------------------------------------------------
cos距離: 0.9349360466003418
最像標題: 欸，有聽過，顯微鏡下的柯學嗎？
最像文章: 

顯微鏡下的柯學

作詞：廢文大師兄
作曲：黃義達

DPP 看批踢踢 在哭泣
洗掉了一些夢 埋葬在高市裡

NPP 看佛來敵 到哪去
好像在預報著 未來的肥皂劇

一封信 一件舊的新聞歸依
誰說誰那麼沒品 拿什麼來回憶

一鬧劇 狠狠演在眼睛裡
愛在老母發言後 已經都沒關係

問自已 沒有柯痞我行不行
顯微鏡裡看柯學 殘酷的放大所有鄉愿的原因

更多更詳盡歌詞 在 ※ Moxxx.com　魔禁歌詞網

遇見P 之後 愛上你 然後 恨透你
原來 柯學是回不去的旅行
親愛的 讓我忘記痞 那些事情 我終於看仔細

深夜裡 捷運站 滑批踢踢
我看著馬路邊 另一位柯學痞
-----
Sent from JPTT on my iPhone

--



/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
